In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import os
train_image_files_path = "../Data/dataset_image/"
valid_image_files_path = "../Data/Validation/"

In [3]:
# Đọc danh sách các thư mục trong đường dẫn train_image_files_path
label = [f for f in os.listdir(train_image_files_path) if os.path.isdir(os.path.join(train_image_files_path, f))]
label.sort()
label

['Asian woolly-necked stork',
 'Clouded leopard',
 'FISHING CAT (PRIONAILURUS VIVERRINUS)',
 'FIVE-BAR SWORDTAIL butterfly',
 'Green peafowl',
 'Hawksbill sea turtle',
 'SAUNDERS_S GULL (SAUNDERSILARUS SAUNDERSI)',
 'Smooth-coated otter',
 'WHALE SHARK (RHINCODON TYPUS)']

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_data_gen = ImageDataGenerator(rescale=1/255)
validation_data_gen = ImageDataGenerator(rescale=1/255)

In [5]:
train_generator = train_data_gen.flow_from_directory(
    train_image_files_path,
    target_size=(200, 200),
    class_mode='sparse'  # Use 'sparse' for integer labels
)

validation_generator = validation_data_gen.flow_from_directory(
    valid_image_files_path,
    target_size=(200, 200),
    class_mode='sparse'  # Use 'sparse' for integer labels
)

Found 1896 images belonging to 9 classes.
Found 1111 images belonging to 9 classes.


In [6]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
import tensorflow as tf
# Define the model
model = Sequential()

#CNN1
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(200,200,3)))
model.add(MaxPooling2D(2,2))

#CNN2
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))

#CNN3
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))

# Flatten the output
model.add(Flatten())
# Hidden layer
model.add(Dense(512, activation='relu'))
# Output layer
model.add(Dense(9, activation='softmax'))  # 9 classes, softmax for probabilities



In [7]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Use sparse categorical crossentropy
              metrics=['acc'])



In [8]:
# Train the model for 100 epochs, with 3 steps per epoch
EPOCHS = 100
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    steps_per_epoch=3,  # Set steps_per_epoch to 3
    validation_data=validation_generator
)


Epoch 1/100


3/3 [==============================] - 24s 10s/step - loss: 9.4623 - acc: 0.1771 - val_loss: 4.7583 - val_acc: 0.2880
Epoch 2/100
3/3 [==============================] - 20s 10s/step - loss: 4.1650 - acc: 0.2396 - val_loss: 1.8972 - val_acc: 0.3888
Epoch 3/100
3/3 [==============================] - 21s 10s/step - loss: 1.9938 - acc: 0.2604 - val_loss: 1.7703 - val_acc: 0.3618
Epoch 4/100
3/3 [==============================] - 21s 10s/step - loss: 1.7686 - acc: 0.2812 - val_loss: 1.5281 - val_acc: 0.5563
Epoch 5/100
3/3 [==============================] - 20s 10s/step - loss: 1.6419 - acc: 0.3438 - val_loss: 1.8030 - val_acc: 0.2682
Epoch 6/100
3/3 [==============================] - 20s 10s/step - loss: 1.5521 - acc: 0.4271 - val_loss: 1.6532 - val_acc: 0.4878
Epoch 7/100
3/3 [==============================] - 20s 9s/step - loss: 1.3059 - acc: 0.5729 - val_loss: 1.3326 - val_acc: 0.5851
Epoch 8/100
3/3 [==============================] - 20s 10s/step - loss: 1.3326 - acc: 0.5

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# Hàm dự đoán
def predict_image(image_path):
    # Load ảnh và chuyển đổi về đúng định dạng
    img = Image.open(image_path)
    img = img.resize((200, 200))
    img_array = np.array(img) / 255.0  # Rescale về 0-1

    # Thêm một chiều cho batch
    img_array = np.expand_dims(img_array, axis=0)

    # Dự đoán
    predictions = model.predict(img_array)
    predicted_label = label[np.argmax(predictions)]
    confidence = np.max(predictions) * 100  # Độ chính xác trong phần trăm

    # Hiển thị ảnh và kết quả dự đoán
    plt.imshow(img)
    plt.title(f'Predicted label: {predicted_label} - Confidence: {confidence:.2f}%')
    plt.axis('off')
    plt.show()

file_path = '../Data/Testing/con_hat.jpg'
predict_image(file_path)



NameError: name 'model' is not defined